In [1]:
import urllib2
import json
import numpy as np
import re
import pandas as pd
import csv
import matplotlib.pyplot as plt 
import collections
from scipy import stats
%matplotlib inline


In [3]:
#  http://api.census.gov/data/2015/acs5/variables.json link to json for api
def get_variable_list_2(YYYY):
    url_code = 'http://api.census.gov/data/' + str(YYYY) + '/acs5/variables.json' 
    json_obj=urllib2.urlopen(url_code)
    var_list = json.load(json_obj)
    return var_list

In [327]:
#var_list = get_variable_list(2015)
var_list_2 = get_variable_list_2(2011)

In [5]:
#get the targeted variables from json api
def var_data(var_list, variable_code):
    variable_dic = {}
    variable_list = []
    for code in var_list['variables']:
        if code.startswith(variable_code) and code.endswith('E'):
            variable_list.append(code)
            variable_dic[code] = var_list['variables'][code]['label']
    return variable_list, variable_dic

In [315]:
def collecting_data_for_specific_year(YYYY, state_code,state_abb, var_list,var_dic): 
    #load a json file containing all the codes for variables in variable list
    #find the code on this website http://api.census.gov/data/2014/acs5/variables.json
    census_api_key = 'b9a9255e89915659ba885b13a0e1d5138ccb6ddc'
    base_url = ' http://api.census.gov/data/'
    final_url = base_url + str(YYYY) + '/acs5?get=' + ','.join(sorted(var_list[:50])) + '&for=county+subdivision:*&in=state:' + str(state_code)+'&key=' + str(census_api_key)
    json_data = urllib2.urlopen(final_url)
    dataframe = json.load(json_data)
    dataframe = pd.DataFrame(dataframe)
    remaining_var_list = var_list[50:]
    while len(remaining_var_list) > 0:
        final_url = base_url + str(YYYY) + '/acs5?get=' + ','.join(sorted(remaining_var_list[:50])) + '&for=county+subdivision:*&in=state:' + str(state_code) + '&key=' + str(census_api_key)
        json_data = urllib2.urlopen(final_url)
        df = json.load(json_data)
        df= pd.DataFrame(df)
        dataframe = pd.concat([dataframe.iloc[:, :-3], df], axis = 1)
        remaining_var_list = remaining_var_list[50:]
    dataframe.columns = dataframe.iloc[0, :]
    dataframe = dataframe.iloc[2:, :]
    
    #replace codes with labels for columns
    for code in dataframe.columns[:-3]:
        dataframe.rename(columns={code: var_dic[code]}, inplace=True)
    #find code that matches county name
    county_dic ={}
    county_sub_div_dic={}
    county = 'http://www2.census.gov/geo/docs/reference/codes/files/st' + str(state_code) + '_' + state_abb + '_cousub.txt'
    county = urllib2.urlopen(county).readlines()
    county = csv.reader(county, delimiter = ',') 
    for line in county:
        county_dic[line[2]] = line[3]
        county_sub_div_dic[line[4]] = line[5]
        
    #remove town or Town city from city name
    for key in county_sub_div_dic:
        county_sub_div_dic[key] = re.sub(' t*T*own\s*\w*', '', county_sub_div_dic[key])
        county_sub_div_dic[key] = re.sub(' city', '', county_sub_div_dic[key])

    dataframe['county'] = dataframe['county'].map(county_dic)
    dataframe['county subdivision'] = dataframe['county subdivision'].map(county_sub_div_dic)
    dataframe.set_index(dataframe['county subdivision'], inplace = True)
    dataframe = dataframe.drop(['county', 'county subdivision', 'state'], axis=1) 
    dataframe = dataframe.fillna(0)
    dataframe = dataframe.applymap(int)
    return dataframe

In [90]:
def collecting_data_for_multiple_years(years, state_code, state_abb, var_list, var_dict):
    y = {}
    for year in years:
        df = collecting_data_for_specific_year(year, state_code,state_abb, var_list,var_dict)
        y[year] = df
    return y

In [92]:
def collect_target_variables(var_dic, list_of_vars, var_data):
    list_of_target_variables = []
    for v in list_of_vars:
        l, d = var_data(var_dic, v)
        list_of_target_variables.append([l,d])
    return list_of_target_variables

In [328]:
target_var = ['B08136','B07404','B07010','B07012', 'B07401','B20001', 'B01001B', 'B25034', 'B19001']

In [329]:
tar_var = collect_target_variables(var_list_2, target_var, var_data)

In [330]:
def df_tar_var(tar_var_list, start_year, end_year):
    dic = {}
    i = 0
    for v in tar_var_list:
        print v[0]
        df = collecting_data_for_multiple_years(range(start_year, end_year + 1), 25, 'ma', v[0], v[1])
        dic[i] = df
        i = i + 1
    return dic

In [331]:
#get the data for each variable from 2011 to 2015
df = df_tar_var(tar_var, 2011, 2015)

[u'B08136_002E', u'B08136_011E', u'B08136_009E', u'B08136_012E', u'B08136_001E', u'B08136_004E', u'B08136_010E', u'B08136_006E', u'B08136_005E', u'B08136_008E', u'B08136_003E', u'B08136_007E']
[u'B07404GPR_001E', u'B07404APR_005E', u'B07404G_003E', u'B07404F_002E', u'B07404A_005E', u'B07404H_001E', u'B07404EPR_003E', u'B07404I_004E', u'B07404I_003E', u'B07404DPR_001E', u'B07404D_001E', u'B07404B_005E', u'B07404BPR_003E', u'B07404FPR_004E', u'B07404DPR_002E', u'B07404C_003E', u'B07404APR_003E', u'B07404G_001E', u'B07404E_002E', u'B07404HPR_005E', u'B07404IPR_005E', u'B07404FPR_001E', u'B07404H_003E', u'B07404IPR_003E', u'B07404CPR_002E', u'B07404H_005E', u'B07404GPR_002E', u'B07404B_003E', u'B07404BPR_005E', u'B07404E_001E', u'B07404A_002E', u'B07404CPR_001E', u'B07404G_004E', u'B07404F_005E', u'B07404I_005E', u'B07404GPR_004E', u'B07404EPR_004E', u'B07404B_004E', u'B07404E_004E', u'B07404D_005E', u'B07404A_001E', u'B07404C_001E', u'B07404APR_001E', u'B07404HPR_003E', u'B07404FPR_003E',

[u'B01001B_017E', u'B01001B_024E', u'B01001B_006E', u'B01001B_014E', u'B01001B_023E', u'B01001B_003E', u'B01001B_030E', u'B01001B_029E', u'B01001B_011E', u'B01001B_019E', u'B01001B_001E', u'B01001B_026E', u'B01001B_009E', u'B01001B_016E', u'B01001B_025E', u'B01001B_004E', u'B01001B_028E', u'B01001B_013E', u'B01001B_020E', u'B01001B_007E', u'B01001B_010E', u'B01001B_018E', u'B01001B_002E', u'B01001B_027E', u'B01001B_015E', u'B01001B_022E', u'B01001B_005E', u'B01001B_031E', u'B01001B_012E', u'B01001B_021E', u'B01001B_008E']
[u'B25034_001E', u'B25034_004E', u'B25034_007E', u'B25034_008E', u'B25034_010E', u'B25034_002E', u'B25034_009E', u'B25034_005E', u'B25034_006E', u'B25034_003E']
[u'B19001E_017E', u'B19001E_003E', u'B19001D_011E', u'B19001F_005E', u'B19001A_001E', u'B19001_004E', u'B19001H_006E', u'B19001I_012E', u'B19001G_012E', u'B19001C_015E', u'B19001B_012E', u'B19001A_009E', u'B19001F_015E', u'B19001_016E', u'B19001I_004E', u'B19001D_002E', u'B19001A_016E', u'B19001G_004E', u'B190

In [336]:
#read zillow csv and group the median list price by mean of each year
def zillow_data(csv_file, state, index_1, index_2):
    z = pd.read_csv(csv_file)
    t = []
    for i in z.iloc[:, 5:].columns.tolist():
        t.append(i[:-3])
    z_ma = z.loc[z['State'] == 'MA'].set_index(['RegionName']).iloc[:, 4:]
    z_ma.columns = t
    z_ma = z_ma.transpose().groupby(t).mean()
    return z_ma



In [198]:
median_list_price = zillow_data('City_MedianListingPricePerSqft_AllHomes.csv', 'MA')

In [187]:
#find common cities in the zillow csv and the collected df from census 
def common_cities_between_zillow_and_df(dic_of_df, zillow, start_year, end_year):
    df_w_zillow ={}
    for key in dic_of_df:
        df_w_zillow[key] = {}
        for year in range(start_year, end_year + 1):
            df = dic_of_df[key][year]
            df['price'] = 0
            for cities in df.index.tolist():
                try:
                    df.loc[cities, 'price'] = zillow.loc[str(year), cities]
                except (KeyError, TypeError, ValueError):
                    continue
            df = df.loc[df['price'] > 0]
            df_w_zillow[key][year] = df
    return df_w_zillow

In [332]:
zillow_df = common_cities_between_zillow_and_df(df, median_list_price, 2011, 2015)

In [123]:
#calculate the percent change in each variable by last years values
def percent_change_from_last_year(dic_of_dic, start_year, end_year):
    new_dic = {}
    for key in dic_of_dic.keys():
        new_dic[key] = {}
        for year in range(start_year+1, end_year+1):
            sub_df = dic_of_dic[key][year].subtract(dic_of_dic[key][year-1], axis = 0)
            percent_change = sub_df.divide(dic_of_dic[key][year-1], axis = 0) * 100
            new_dic[key][year] = percent_change
    return new_dic

In [333]:
percent_change = percent_change_from_last_year(zillow_df, 2011, 2015)

In [386]:
#identify features that may correlate with price change
def features(dic_of_df, start_year, end_year):
    for key in dic_of_df:
        for year in range(start_year,end_year+1):
            df = dic_of_df[key][year]
            print df.corr(method='pearson').loc[:,'price'].sort_values(ascending = False)[1:10]

features(percent_change, 2012, 2015)      
        

0
Public transportation (excluding taxicab):!!Railroad or ferryboat                                                          0.063459
Car, truck, or van:!!Carpooled:!!In 2-person carpool                                                                       0.046027
Public transportation (excluding taxicab):!!Streetcar or trolley car (carro publico in Puerto Rico), subway or elevated    0.005160
Car, truck, or van:!!Drove alone                                                                                           0.005038
Car, truck, or van:                                                                                                        0.002295
Aggregate travel time to work (in minutes):                                                                               -0.004002
Car, truck, or van:!!Carpooled:                                                                                           -0.010237
Taxicab, motorcycle, bicycle, or other means                              

0
$40,000 to $44,999      1.000000
$10,000 to $14,999      1.000000
$100,000 to $124,999    0.950302
$30,000 to $34,999      0.583107
$45,000 to $49,999      0.535026
$150,000 to $199,999    0.528008
$50,000 to $59,999      0.475527
$15,000 to $19,999      0.310447
$25,000 to $29,999      0.305046
Name: price, dtype: float64
0
$35,000 to $39,999      1.000000
$10,000 to $14,999      1.000000
$20,000 to $24,999      1.000000
$75,000 to $99,999      0.999879
$45,000 to $49,999      0.974390
$75,000 to $99,999      0.780656
$35,000 to $39,999      0.549341
$125,000 to $149,999    0.479554
$200,000 or more        0.419836
Name: price, dtype: float64
